In [2]:
from sumo_deeprl import SumoDeepRl
import os
import traci
from stable_baselines3.dqn.dqn import DQN

### Init Sumo-DeepRl class

In [2]:
sumo = SumoDeepRl('saint_paul/simulation_saintpaul.net.xml')

### Generate Trips (optional)

In [15]:
output_route = './saint_paul/trips/1hour/'
route_details = {'n': ['-E2', 6.5], 
                 's': ['-E0', 5.5],
                 'e': ['-E3', 3],
                 'w': ['-E1', 2]}

for cars in [1000, 2000, 3000, 4000, 5000]:
  sumo.create_routes(route_details=route_details, time=3600, total_cars=cars,output_dir=output_route)

Create Success
Create Success
Create Success
Create Success
Create Success


### Simulation

In [8]:
sumo.simulation(route_file='./saint_paul/trips/1hour/1000.rou.xml', useGui=True)
sumo.analyze_fixed(dir='./saint_paul/trips/1hour',vphs=[1000], output_dir='./saint_paul/results/1hour/simulation', useGui=False)

1000 cars: waiting time 55726.0, clear time: 3791.0


In [ ]:
traci.close()

### Model

In [9]:
env = sumo.create_environment(route_file='./saint_paul/trips/1hour/3000.rou.xml', yellow_time=5)

In [10]:
model = DQN(
  env=env,
  policy="MlpPolicy",
  learning_starts=0,
  train_freq=1,
  exploration_initial_eps=0.05,
  exploration_final_eps=0.01,
  verbose=1
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
total_timesteps = 10000
model.learn(total_timesteps=total_timesteps, reset_num_timesteps=False)
out_dir = "./saint_paul/model/1hour/"
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
fullname = os.path.join(out_dir, '1000')   
model.save(fullname)
env.close()

### Result from model

In [12]:
train_model = DQN.load('./saint_paul/model/1hour/1000.zip', env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
sumo.predict(route_file='./saint_paul/trips/1hour/3000.rou.xml', model=train_model, useGui=True, yellow_time=5)

(251424.0, 7326.0)

In [ ]:
sumo.analyze_predict(model=train_model,dir='./saint_paul/trips/1hour',vphs=[1000,2000,3000,4000,5000],out_dir='./saint_paul/results/1hour/predict', yellow_time=5)

In [3]:
traci.close()

FatalTraCIError: Not connected.